In [20]:
!pip install -q -r requirements.txt
import pandas as pd
import numpy as np
import matplotlib as plt
import datapackage
#import tabula
#from tabula import read_pdf

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


# Data Cleaning 

## Country data 
Additional data required to join WDI & IFS data on common country code

In [ ]:

WOE_Dataset = pd.read_csv('IMF_country_codes.csv')
def WEO_country(WOE_Dataset):
    # Get ride of unused data
    WOE_Dataset.drop(columns= WOE_Dataset.columns[4:] , inplace=True)
    #rename columns
    old_columns = WOE_Dataset.columns
    New_columns = WOE_Dataset.iloc[0].to_list()
    rename_map = dict(zip(old_columns, New_columns))
    WOE_Dataset.rename(columns = rename_map, inplace = True)

    #Drop column 0 - the true header
    WOE_Dataset = WOE_Dataset.drop(0).copy()
    return WOE_Dataset 

In [ ]:
cntry_map = pd.read_csv('IMF_country_codes.csv')
cntry_map = CountrieWEO_country(cntry_map)

,IMF Code,ISO Code,Country,Currency
1,512,AFG,Afghanistan,Afghan Afghani
2,914,ALB,Albania,Albanian lek
3,612,DZA,Algeria,Algerian dinar
4,614,AGO,Angola,Angolan kwanza
5,311,ATG,Antigua and Barbuda,Eastern Caribbean dollar
...,...,...,...,...
185,299,VEN,Venezuela,Venezuelan bol�var fuerte
186,582,VNM,Vietnam,Vietnamese dong
187,474,YEM,Yemen,Yemeni rial
188,754,ZMB,Zambia,Zambian kwacha


### World Develpment Indicators (WDI) Data - The World Bank
Notes on outstanding work: 1 - Make Data DateTime Object. 2 - concat the Indicator code to the title name

In [ ]:
wdi_sample = pd.read_csv('WDI - sample.csv')

In [ ]:

#Clean titles
#Create Dictionary of column header mapping & use rename function
old_columns = wdi_sample.columns[4::]
yr_columns = wdi_sample.columns[4::].str[:4]
yr_columns_dict = dict(zip(old_columns, yr_columns))
wdi_sample.rename(columns = yr_columns_dict, inplace = True)

#Country Selection
cntrys = ['South Africa', 'Afghanistan']
wdi_sample[wdi_sample['Country Name'].isin(cntrys)]

#Reform Pivotted Data
wdi_sample.drop(['Series Code'], axis=1, inplace = True)
wdi_sample = wdi_sample.melt(id_vars=['Country Name','Country Code','Series Name'], var_name='Year', value_name='Value' ).copy()
# Note, this won't work unless you've got selected countries! 
wdi_sample = wdi_sample.pivot(index=['Year', 'Country Name' ,'Country Code'], columns='Series Name', values='Value').reset_index().copy()

ValueError: Index contains duplicate entries, cannot reshape

In [ ]:
wdi_sample

#### World Development Indicators data Cleaning Function



In [ ]:
def clean_wdi_data(dataset, countries, date_from = 1980, date_to = 2002 ):
    '''
    dataset {df}: WDI Data sample
    countries {List}: List of countries to be inclided in Dataset
    date_from {int}: Lowest year - default 1980
    date_to {int}: Highest Yeat - default 2020

    Output{df}: Year, Country Name , Country Code, [Series Names]

    '''
    #Clean titles
    #Create Dictionary of column header mapping & use rename function
    old_columns = dataset.columns[4::]
    yr_columns = dataset.columns[4::].str[:4]
    yr_columns_dict = dict(zip(old_columns, yr_columns))
    dataset.rename(columns = yr_columns_dict, inplace = True)
    
    #Country Selection
    dataset = dataset[dataset['Country Name'].isin(countries)].copy()
    
    #Reform Pivotted Data
    dataset.drop([ 'Series Code'], axis=1, inplace = True)
    dataset = dataset.melt(id_vars=['Country Name','Country Code','Series Name'], var_name='Year', value_name='Value' ).copy()
    dataset = dataset.pivot(index=['Year', 'Country Name', 'Country Code'], columns='Series Name', values='Value').reset_index().copy()

    return dataset

In [ ]:
test = pd.read_csv('WDI - sample.csv')
hun = ['Hungary', 'South Africa']
clean_wdi_data(test, hun, date_from = 1980, date_to = 2002 )

Series Name,Year,Country Name,Country Code,Broad money growth (annual %),Broad money to total reserves ratio,Gross capital formation (constant 2010 US$),Real effective exchange rate index (2010 = 100)
0,1980,Hungary,HUN,..,..,..,..
1,1980,South Africa,ZAF,22.8507513885544,5.47953147000172,41470623355.0728,165.188043881133
2,1981,Hungary,HUN,..,..,..,..
...,...,...,...,...,...,...,...
79,2019,South Africa,ZAF,6.11024523226612,4.73151814615898,81243117953.9007,78.1427521961595
80,2020,Hungary,HUN,20.8940730816157,..,..,85.4569190386896
81,2020,South Africa,ZAF,9.42732646118054,..,..,70.9184570461496


### International Financial Statistics (IFS DB) data - International Monetary Fund
Notes on outstanding work: 1 - Make Data DateTime Object. 2 - concat the Indicator code to the title name. 3 - Need to Edit the Indicators being used. 

| Field      | Indicator Code | Source    |
| :---        |    :----:   |    :---: |
| Header      | Title       | Here's this   |
| Paragraph   | Text        | And more      |

In [27]:
def clean_IFS_data(dataset, countries, frequency, date_from, Series_Codes , date_to):
    '''
    dataset {df}: WDI Data sample
    countries {List}: List of countries to be inclided in Dataset
    date_from {int}: Lowest year - default 1980
    date_to {int}: Highest Yeat - default 2020

    Output{df}: 'Year', 'Country Name', 'Country Code', [Series Names]

    '''
    #Country Selection
    dataset = dataset[dataset['Country Name'].isin(countries)].copy()   

    #Concat Base year & Drop Base Year Column
    dataset['Indicator Name'] = np.where(dataset['Base Year'].isnull(),dataset['Indicator Name'], dataset['Indicator Name'] + " (" + dataset['Base Year'] +")").copy()
    
    # Filter by then concat the Indicator Code
    dataset = dataset[dataset['Indicator Code'].isin(Series_Codes)].copy()
    dataset['Indicator Name'] = dataset['Indicator Name'] + " - " + dataset['Indicator Code']

    dataset = dataset[dataset['Attribute']!='Status'].copy()
    
    dataset.drop(['Base Year', 'Indicator Code', 'Attribute'], axis=1, inplace = True)


    #Melt Data 
    dataset = dataset.melt(id_vars=['Country Name','Country Code','Indicator Name'], var_name='Year', value_name='Value' ).copy()

    #Frequency - Annual - Done
    if frequency == 'Y':
        non_ann = ['M', 'Q']
        dataset = dataset[~dataset.Year.str.contains('|'.join(non_ann))].copy()
    elif frequency == 'Q':
        dataset = dataset[dataset.Year.str.contains('Q')].copy()
    elif frequency == 'M':
        dataset = dataset[dataset.Year.str.contains('M')].copy()

    #Pivot Indicators to Columns
    dataset['Value'] = dataset['Value'].astype(float)
    dataset = dataset.groupby(['Year', 'Country Name', 'Country Code', 'Indicator Name']).agg({'Value':'sum'}).reset_index().copy()
    dataset = dataset.pivot(index=['Year', 'Country Name', 'Country Code'], columns='Indicator Name', values='Value').reset_index().copy()

    return dataset

IMF Load Function Test:

In [16]:
IFS_DB =  pd.read_csv('IFS_timeSeries_1980_2020.csv')
IFS_DB.head()

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (5,6,7,8,10,13,14,15,16,18,19,21,22,23,24,25,26,27,28,31,32,35,38,39,40,41,48,49,50,52,54,55,56,57,58,59,60,65,67,69,72,73,74,75,77,78,79,80,81,82,83,86,87,88,89,90,91,92,93,95,96,99,100,101,102,103,106,107,108,109,110,112,113,115,116,120,121,124,125,126,129,130,133,134,137,138,140,141,142,143,146,148,149,150,151,153,154,155,157,158,159,160,161,162,164,165,167,168,171,175,176,177,180,181,184,186,188,191,192,193,194,197,198,200,201,202,204,205,206,207,209,210,211,212,214,215,217,218,219,220,222,223,224,226,227,228,229,230,231,232,233,235,236,237,238,239,240,241,242,243,244,245,248,249,251,252,253,254,255,256,259,260,261,262,263,265,267,269,270,271,272,276,277,278,279,280,282,283,286,287,292,293,294,295,296,297,303,304,305,306,307,309,310,311,312,313,316,320,321,323,324,326,328,329,330,333,334,335,337,338,341,343,344,345,346,347,349,350,351,353,354,355,356,357,358,359,3

,Country Name,Country Code,Indicator Name,Indicator Code,Attribute,1980,1980Q1,1980M1,1980M2,1980M3,...,2020M12,2021,2021Q1,2021M1,2021M2,2021M3,2021Q2,2021M4,2021M5,Base Year
0,"Afghanistan, Islamic Rep. of",512,"Fund Accounts, UFC & Loans, SDRs",HPU_XDR,Value,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Afghanistan, Islamic Rep. of",512,"Fund Accounts, UFC & Loans, SDRs",HPU_XDR,Value,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,281230000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Afghanistan, Islamic Rep. of",512,"Fund Accounts, UFC & Loans, SDRs",HPU_XDR,Value,NaN,NaN,0.0,0.0,0.0,...,281230000.0,NaN,NaN,281230000.0,281230000.0,281230000.0,NaN,278830000.0,NaN,NaN
3,Advanced Economies,110,"Fund Accounts, Outstanding Loans, SDRs",HPL_XDR,Value,30600000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Advanced Economies,110,"Fund Accounts, Outstanding Loans, SDRs",HPL_XDR,Value,NaN,37550000.0,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
Countries = ['Thailand', 'South Africa', 'Turkey']

IMF_Indicators =['FILR_PA'
                ,'RAXGFX_USD'
                ,'NM_XDC' 
                ,'TMG_CIF_USD'
                ,'TXG_FOB_USD'
                ,'IAP_BP6_USD']
X = clean_IFS_data(IFS_DB, countries = Countries, date_from = 1980, Series_Codes = IMF_Indicators , date_to = 2002, frequency = 'Q' )
X.head(100)

Indicator Name,Year,Country Name,Country Code,"Assets, Portfolio Investment, US Dollars - IAP_BP6_USD","External Balance of Goods and Services, Imports of Goods and Services, Nominal, Domestic Currency - NM_XDC","External Trade, Goods, Value of Exports, Free on Board (FOB), US Dollars - TXG_FOB_USD","External Trade, Goods, Value of Imports, Cost, Insurance, Freight (CIF), US Dollars - TMG_CIF_USD","Financial, Interest Rates, Lending Rate, Percent per annum - FILR_PA","International Liquidity, Total Reserves excluding Gold, Foreign Exchange, US Dollars - RAXGFX_USD"
0,1980Q1,South Africa,199,0.0,0.0,6.502265e+09,3.607429e+09,9.500000,5.640000e+08
1,1980Q1,Thailand,578,0.0,0.0,1.928077e+09,2.140478e+09,16.000000,1.636000e+09
2,1980Q1,Turkey,186,0.0,0.0,7.140100e+08,1.398400e+09,NaN,9.560000e+08
3,1980Q2,South Africa,199,0.0,0.0,5.783546e+09,4.908747e+09,9.500000,6.690000e+08
4,1980Q2,Thailand,578,0.0,0.0,1.551523e+09,2.254750e+09,16.833333,2.136000e+09
...,...,...,...,...,...,...,...,...,...
95,1987Q4,Turkey,186,0.0,4905000.0,3.321600e+09,4.762600e+09,NaN,1.730000e+09
96,1988Q1,South Africa,199,0.0,0.0,5.321618e+09,4.508897e+09,13.333333,4.730000e+08
97,1988Q1,Thailand,578,0.0,0.0,3.507266e+09,4.705675e+09,11.500000,4.419000e+09
98,1988Q1,Turkey,186,0.0,4450000.0,2.802400e+09,3.492700e+09,NaN,1.663000e+09


Useful code to find data type for each value in the dataframe

## World Bank API 

In [34]:
import pandas as pd
import world_bank_data as wb
pd.set_option('display.max_rows', 6)

In [ ]:
wb.get_topics()

In [43]:
pd.DataFrame(wb.get_series('FM.LBL.MQMY.ZG', date='2010:2018', id_or_value='id', simplify_index=True)).reset_index()

#FM.LBL.MQMY.IR.ZS

ValueError: The indicator was not found. It may have been deleted or archived.
url=http://api.worldbank.org/v2/country/all/indicator/FM.LBL.MQMY.ZG
params={'date': '2010:2018', 'format': 'jsonstat', 'per_page': 20000}

In [36]:
wb.search_indicators('M2')

,name,unit,source,sourceNote,sourceOrganization,topics
id,,,,,,
8.3.1_MDG.PASEC.CM2.FRE,"PASEC in Madagascar, CM2, French (mean score)",,Global Partnership for Education,Mean score calculated for the results of the P...,Data were collected from national and other pu...,
8.3.11_GIN.LEAR.TEST.CEPE.MEAN,National assessment at the end of primary (CEP...,,Global Partnership for Education,Mean score calculated for the results of the n...,Data were collected from national and other pu...,
8.3.12_NER.LEAR.TEST.CM2.FR.UNDERMIN,National assessment for learning outcomes in N...,,Global Partnership for Education,Students under minimal competency in the resul...,Data were collected from national and other pu...,
...,...,...,...,...,...,...
SH.STA.OWAD.MA.ZS,"Prevalence of overweight, male (% of male adults)",,Health Nutrition and Population Statistics,Prevalence of overweight male adults is the pe...,"World Health Organization, Global Health Obser...",
SH.STA.OWAD.ZS,Prevalence of overweight (% of adults),,Health Nutrition and Population Statistics,Prevalence of overweight adults is the percent...,"World Health Organization, Global Health Obser...",
SPI.D5.2.8.FINA,Compilation of government finance statistics,,Statistical Performance Indicators (SPI),Compilation of government finance statistics r...,"Statistical Performance Indicators, The World ...",


## IMF API

In [10]:
# Import libraries
import requests
import pandas as pd

# URL for the IMF JSON Restful Web Service, 
# IFS database, and Australian export prices series
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/IFS/Q.TH.IAP_BP6_USD.?startPeriod=1980&endPeriod=2020'

# Get data from the above URL using the requests package
data = requests.get(url).json()

# Load data into a pandas dataframe
auxp = pd.DataFrame(data['CompactData']['DataSet']['Series']['Obs'])

# Show the last five observiations
auxp.tail(100)

,@TIME_PERIOD,@OBS_STATUS,@OBS_VALUE
0,2005-Q1,B,NaN
1,2012-Q1,NaN,25319.986632403
2,2012-Q2,NaN,25947.2903294317
3,2012-Q3,NaN,26031.8845764803
4,2012-Q4,NaN,29279.5618361294
5,2013-Q1,NaN,30740.9321639172
6,2013-Q2,NaN,28930.3027688052
7,2013-Q3,NaN,28569.9034566963
8,2013-Q4,NaN,29965.7793022461
9,2014-Q1,NaN,31554.5977277211


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5afecb5f-7d28-4b37-ae51-8d94c0fa9663' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>